In [3]:
#!/usr/bin/env python

import os
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# ============================================================
# CONFIG
# ============================================================

STRAT_EQUITY_FILE = "./13-trading_output_regression_only/13-equity_curve.parquet"

OUT_DIR = "./16-monte_carlo_output_no_spy"
os.makedirs(OUT_DIR, exist_ok=True)

N_SIMS = 10000
BLOCK_SIZE = 25
SEED = 123
np.random.seed(SEED)

# ============================================================
# METRICS
# ============================================================

def cagr(series):
    series = np.asarray(series)
    if len(series) < 2:
        return np.nan
    return (series[-1] / series[0]) ** (252/len(series)) - 1

def sharpe(returns):
    if returns.std(ddof=1) == 0:
        return 0
    return np.sqrt(252) * np.mean(returns) / np.std(returns, ddof=1)

def sortino(returns):
    downside = returns[returns < 0]
    if downside.std(ddof=1) == 0:
        return 0
    return np.sqrt(252) * returns.mean() / downside.std(ddof=1)

def max_drawdown(values):
    arr = np.asarray(values)
    peaks = np.maximum.accumulate(arr)
    dd = arr / peaks - 1
    return dd.min()

def calmar(cagr_val, dd):
    if dd == 0:
        return np.nan
    return cagr_val / abs(dd)

# ============================================================
# BLOCK BOOTSTRAP
# ============================================================

def block_bootstrap(returns, n, block_size):
    out = []
    L = len(returns)

    while len(out) < n:
        start = np.random.randint(0, L - block_size)
        block = returns[start:start+block_size]
        out.extend(block)

    return np.array(out[:n])

# ============================================================
# LOAD DATA
# ============================================================

print("=== Loading equity curve ===")

equity = pd.read_parquet(STRAT_EQUITY_FILE)
equity["date"] = pd.to_datetime(equity["date"])
equity = equity.sort_values("date")

equity["strat_ret"] = equity["portfolio_value"].pct_change()
equity = equity.dropna()

rets = equity["strat_ret"].values
n_days = len(rets)

print(f"Loaded {n_days} daily return observations.\n")

# ============================================================
# TRUE METRICS
# ============================================================

true_curve = equity["portfolio_value"].values
true_cagr = cagr(true_curve)
true_sharpe = sharpe(rets)
true_sortino = sortino(rets)
true_maxdd = max_drawdown(true_curve)
true_calmar = calmar(true_cagr, true_maxdd)

print("=== TRUE STRATEGY RESULTS ===")
print(f"CAGR:       {true_cagr:.4f}")
print(f"Sharpe:     {true_sharpe:.4f}")
print(f"Sortino:    {true_sortino:.4f}")
print(f"MaxDD:      {true_maxdd:.4f}")
print(f"Calmar:     {true_calmar:.4f}\n")

# ============================================================
# MONTE CARLO SIMULATION
# ============================================================

print("=== Running Monte Carlo simulations ===")

sim_cagrs = []
sim_sharpes = []
sim_sortinos = []
sim_dds = []
sim_calmars = []

for _ in range(N_SIMS):
    sim_rets = block_bootstrap(rets, n_days, BLOCK_SIZE)
    sim_curve = (1 + sim_rets).cumprod()

    c = cagr(sim_curve)
    s = sharpe(sim_rets)
    so = sortino(sim_rets)
    dd = max_drawdown(sim_curve)
    ca = calmar(c, dd)

    sim_cagrs.append(c)
    sim_sharpes.append(s)
    sim_sortinos.append(so)
    sim_dds.append(dd)
    sim_calmars.append(ca)

sim_cagrs = np.array(sim_cagrs)
sim_sharpes = np.array(sim_sharpes)
sim_sortinos = np.array(sim_sortinos)
sim_dds = np.array(sim_dds)
sim_calmars = np.array(sim_calmars)

# ============================================================
# SUMMARY STATISTICS
# ============================================================

summary = {
    "avg_cagr": sim_cagrs.mean(),
    "median_cagr": np.median(sim_cagrs),
    "std_cagr": sim_cagrs.std(),

    "avg_sharpe": sim_sharpes.mean(),
    "avg_sortino": sim_sortinos.mean(),
    "avg_maxdd": sim_dds.mean(),
    "avg_calmar": np.nanmean(sim_calmars),

    "cagr_5pct": np.percentile(sim_cagrs, 5),
    "cagr_95pct": np.percentile(sim_cagrs, 95),

    "prob_cagr_below_zero": np.mean(sim_cagrs < 0),
    "prob_ruin": np.mean((1 + rets).cumprod()[-1] < 1),  # finishes below initial

    "prob_dd_worse_than_actual": np.mean(sim_dds < true_maxdd),
    "prob_cagr_below_actual": np.mean(sim_cagrs < true_cagr),
    "prob_sharpe_below_actual": np.mean(sim_sharpes < true_sharpe),
}

print("\n=== SUMMARY OF MONTE CARLO RUNS ===")
for k, v in summary.items():
    print(f"{k:25s}: {v:.4f}")

# ============================================================
# SAVE RESULTS
# ============================================================

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

df_out = pd.DataFrame({
    "sim_cagr": sim_cagrs,
    "sim_sharpe": sim_sharpes,
    "sim_sortino": sim_sortinos,
    "sim_maxdd": sim_dds,
    "sim_calmar": sim_calmars,
})
df_out.to_csv(os.path.join(OUT_DIR, f"16-monte_carlo_results_{timestamp}.csv"), index=False)

pd.DataFrame([summary]).to_csv(os.path.join(OUT_DIR, f"16-summary_stats_{timestamp}.csv"), index=False)

print("\nSaved results + summary stats to:", OUT_DIR)
print("\n=== COMPLETE ===")


=== Loading equity curve ===
Loaded 6772 daily return observations.

=== TRUE STRATEGY RESULTS ===
CAGR:       0.1666
Sharpe:     0.8199
Sortino:    1.0848
MaxDD:      -0.5522
Calmar:     0.3017

=== Running Monte Carlo simulations ===

=== SUMMARY OF MONTE CARLO RUNS ===
avg_cagr                 : 0.1672
median_cagr              : 0.1665
std_cagr                 : 0.0468
avg_sharpe               : 0.8223
avg_sortino              : 1.0958
avg_maxdd                : -0.4499
avg_calmar               : 0.3980
cagr_5pct                : 0.0918
cagr_95pct               : 0.2445
prob_cagr_below_zero     : 0.0000
prob_ruin                : 0.0000
prob_dd_worse_than_actual: 0.1363
prob_cagr_below_actual   : 0.5003
prob_sharpe_below_actual : 0.4937

Saved results + summary stats to: ./16-monte_carlo_output_no_spy

=== COMPLETE ===
